# CONV_LSTM

# Imports


In [ ]:
%%capture
!pip install keras-self-attention
!pip install celluloid

In [ ]:
import os
from glob import glob
import cv2
import xarray as xr
import numpy as np
from tqdm import tqdm
import os.path
import requests
from bs4 import BeautifulSoup
from joblib import Parallel, delayed
import sys
import matplotlib.pyplot as plt
import io
import imageio
from IPython.display import Image, display
from ipywidgets import widgets, Layout, HBox
from celluloid import Camera
import datetime
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf

from tensorflow import keras
from keras import layers
from keras import backend as K
from keras.callbacks import CSVLogger, ModelCheckpoint
from keras_self_attention import SeqSelfAttention

In [ ]:
import logging
tf.get_logger().setLevel(logging.ERROR)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Preparation

In [ ]:
def get_seqed(data, seq_len):
        stacked = np.stack(data)
        seqed_data = []
        for i in range(stacked.shape[0]-seq_len):
            seqed_data.append(stacked[i:i+seq_len])
        return np.stack(seqed_data)

def get_data_for_conv_lstm(dataset):
    # Split into train and validation sets using indexing to optimize memory.
    indexes = np.arange(dataset.shape[0])
    np.random.shuffle(indexes)
    train_val_index = indexes[: int(0.8 * dataset.shape[0])]
    test_index = indexes[int(0.8 * dataset.shape[0]) :]
    train_val_dataset = dataset[train_val_index]
    test_dataset = dataset[test_index]

    train_val_index = np.arange(train_val_dataset.shape[0])
    train_index = train_val_index[: int(0.9 * train_val_dataset.shape[0])]
    val_index = train_val_index[int(0.9 * train_val_dataset.shape[0]):]
    train_dataset = train_val_dataset[train_index]
    val_dataset = train_val_dataset[val_index]

    # Normalize the data to the 0-1 range.
    train_dataset[train_dataset < 0] = 0
    train_dataset[train_dataset > 255] = 255
    val_dataset[val_dataset < 0] = 0
    val_dataset[val_dataset > 255] = 255
    test_dataset[test_dataset < 0] = 0
    test_dataset[test_dataset > 255] = 255

    min_train = np.min(train_dataset)
    max_train = np.max(train_dataset)

    print("Min train: {}, Max train: {}".format(min_train, max_train))

    train_dataset = train_dataset
    val_dataset = val_dataset
    test_dataset = test_dataset

    def create_shifted_frames(data):
      x = data[:, 0 : data.shape[1] - 1, :, :]
      y = data[:, 1: data.shape[1], :, :]
      return x, y

    x_train, y_train = create_shifted_frames(train_dataset)
    x_val, y_val = create_shifted_frames(val_dataset)
    x_test, y_test = create_shifted_frames(test_dataset)

    print("Training Dataset Shapes: " + str(x_train.shape) + ", " + str(y_train.shape))
    print("Validation Dataset Shapes: " + str(x_val.shape) + ", " + str(y_val.shape))

    return x_train, y_train, x_val, y_val, x_test, y_test, train_dataset, val_dataset, test_dataset

In [ ]:
# convections = np.load('/content/drive/MyDrive/Licenta/np_data/np_data.npy')
# convections = np.float32(convections)
# convections = get_seqed(convections, 9)
x_train, y_train, x_val, y_val, x_test, y_test, train_dataset, val_dataset, test_dataset = get_data_for_conv_lstm(np.load('/content/drive/MyDrive/Licenta/np_data/np_data_seq_9_float32.npy'))

Min train: 0.0, Max train: 255.0
Training Dataset Shapes: (2057, 8, 120, 160, 3), (2057, 8, 120, 160, 3)
Validation Dataset Shapes: (229, 8, 120, 160, 3), (229, 8, 120, 160, 3)


# Model


In [ ]:
loss_tracker = keras.metrics.Mean(name="loss")
mse_metric = keras.metrics.MeanSquaredError(name="mse")
mae_metric = keras.metrics.MeanAbsoluteError(name="mae")

beta = 0.01
delta = 1.35

class CustomModel(keras.Model):
  def test_step(self, data):
    x, y = data
    y_pred = self(x, training=False)
    self.compiled_loss(y[:, -1, :, :, :], y_pred[:, -1, :, :, :], regularization_losses=self.losses)
    self.compiled_metrics.update_state(y, y_pred)
    return {m.name: m.result() for m in self.metrics}

  def train_step(self, data):
    x, y = data

    with tf.GradientTape() as tape:
      y_pred = self(x, training=True)
      # loss = tf.keras.losses.Huber(delta=delta)(y,y_pred)
      loss = keras.losses.mean_squared_error(y, y_pred)
      # keras.losses.mean_absolute_error(y, y_pred)
      # loss = tf.nn.l2_loss(tf.subtract(y_pred, y))

    trainable_vars = self.trainable_variables
    gradients = tape.gradient(loss, trainable_vars)

    self.optimizer.apply_gradients(zip(gradients, trainable_vars))

    loss_tracker.update_state(loss)
    mse_metric.update_state(y, y_pred)
    mae_metric.update_state(y, y_pred)

    return {"loss": loss_tracker.result(), "mse": mse_metric.result(),  "mae": mae_metric.result()}

  # property
  # def metrics(self):
  #   return [loss_tracker, mse_metric, mae_metric]

In [ ]:
def build_conv_lstm(input_shape):
    def input_layer():
      return layers.Input(shape=(None, *input_shape[2:]), name="conv_lstm_input")
    
    def convLSTM2D_block(filters: int, kernel_size, padding, strides, return_sequences, activation, normalized, layer_number):
      model = keras.Sequential(name="convLSTM2D_{}_block_{}".format(filters, layer_number))
      model.add(layers.ConvLSTM2D(
          filters=filters,
          kernel_size=kernel_size,
          padding=padding,
          strides=strides, 
          return_sequences=return_sequences,
          activation=activation,
          activity_regularizer='l1'
          ))
      if normalized:
        model.add(layers.BatchNormalization())
      
      return model
    
    def timeDistributed_block(filters, kernel_size, padding, strides, activation, normalized, layer_number):
      model = keras.Sequential(name="timeDistributed_{}_block_{}".format(filters, layer_number))
      model.add(layers.TimeDistributed(
          layers.Conv2DTranspose(
            filters=filters,
            kernel_size=kernel_size,
            padding=padding,
            strides=strides,
            activation=activation
      )))
      if normalized:
          model.add(layers.BatchNormalization())
      
      return model

    def conv3D_block(filters, kernel_size, activation, padding, layer_number):
      model = keras.Sequential(name="conv3D_{}_block_{}".format(filters, layer_number))
      model.add(layers.Conv3D(
          filters=filters,
          kernel_size=kernel_size,
          activation=activation,
          padding=padding
      ))

      return model

    def create_model():
      inputs = input_layer()

      x = inputs
      x = convLSTM2D_block(filters=128, kernel_size=(5,5), padding="same", strides=(1,1), return_sequences=True, activation='relu', normalized=True, layer_number=1)(x)
      x = convLSTM2D_block(filters=256, kernel_size=(5,5), padding="same", strides=(2,2), return_sequences=True, activation='relu', normalized=True, layer_number=2)(x)
      x = convLSTM2D_block(filters=256, kernel_size=(3,3), padding="same", strides=(1,1), return_sequences=True, activation='relu', normalized=True, layer_number=3)(x)
      x = convLSTM2D_block(filters=512, kernel_size=(1,1), padding="same", strides=(2,2), return_sequences=True, activation='relu', normalized=True, layer_number=4)(x)
      x = timeDistributed_block(filters=256, kernel_size=(2,2), padding="same", strides=(2,2), activation='relu', normalized=True, layer_number=5)(x)
      x = timeDistributed_block(filters=128, kernel_size=(2,2), padding="same", strides=(2,2), activation='relu', normalized=False, layer_number=6)(x)
      x = conv3D_block(filters=3, kernel_size=(1,1,1), padding="same",activation='relu', layer_number=7)(x)

      outputs = layers.Add(name="conv_lstm_output")([inputs, x])

      return CustomModel(inputs=inputs, outputs=outputs, name="conv_lstm")
    
    return create_model()

In [ ]:
model = build_conv_lstm(x_train.shape)
# model = tf.keras.models.load_model('/content/drive/MyDrive/Licenta/models/ultimate_weamyl-lstm_best_model_20220320193323.hdf5', custom_objects={'CustomModel': CustomModel})

# Train

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001))

now = int(datetime.datetime.now().strftime("%Y%m%d%H%M%S"))
# reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_delta=0.1, verbose=1)
csv_logger = CSVLogger('/content/drive/MyDrive/Licenta/logs/weamyl-lstm_log_{}.csv'.format(now), append=True, separator=',')
checkpoint = ModelCheckpoint('/content/drive/MyDrive/Licenta/models/weamyl-lstm_best_model_{}.hdf5'.format(now), monitor='loss', verbose=1, save_best_only=True, mode='auto', period=1)
epochs = 200
batch_size = 1


In [ ]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, callbacks=[checkpoint, csv_logger])

Epoch 1/200
2057/2057 [==============================] - ETA: 0s - loss: 95.5197 - mse: 95.5197 - mae: 4.4380
Epoch 1: loss improved from inf to 95.04255, saving model to /content/drive/MyDrive/Licenta/models/weamyl-lstm_best_model_20220320193323.hdf5
2057/2057 [==============================] - 1808s 870ms/step - loss: 95.5195 - mse: 95.5195 - mae: 4.4380
Epoch 2/200
2057/2057 [==============================] - ETA: 0s - loss: 93.9891 - mse: 94.4294 - mae: 4.4093
Epoch 2: loss improved from 95.04255 to 93.98914, saving model to /content/drive/MyDrive/Licenta/models/weamyl-lstm_best_model_20220320193323.hdf5
2057/2057 [==============================] - 1789s 870ms/step - loss: 93.9891 - mse: 94.4292 - mae: 4.4093
Epoch 3/200
2057/2057 [==============================] - ETA: 0s - loss: 93.0248 - mse: 93.4334 - mae: 4.3874
Epoch 3: loss improved from 93.98914 to 93.02477, saving model to /content/drive/MyDrive/Licenta/models/weamyl-lstm_best_model_20220320193323.hdf5
2057/2057 [=========

# Testing

In [ ]:
# model = build_conv_lstm(x_train.shape)
model = tf.keras.models.load_model('/content/drive/MyDrive/Licenta/models/ultimate_weamyl-lstm_best_model_20220320193323.hdf5', custom_objects={'CustomModel': CustomModel})

In [ ]:
def nmae(y_true, y_pred):
  mae = keras.losses.MeanAbsoluteError()(y_true, y_pred)
  nmae = 100 * mae / (tf.reduce_max(y_true) - tf.reduce_min(y_true))
  return nmae

In [ ]:
model.compile(loss="mae", metrics=["mae", "mse", nmae])

model.evaluate(x_test, y_test, batch_size=8)

72/72 [==============================] - 167s 2s/step - loss: 345723.6250 - mae: 4.3061 - mse: 88.0229 - nmae: 1.6881


[345723.625, 4.306107044219971, 88.0229263305664, 1.6881005764007568]

# Frame Prediction Visualizations

In [ ]:
all = []
dx, dy = 30,30
grid_color = [255,255,255]

# max_preds = x_test.shape[0]
max_preds = 25
for img, truth in zip(x_test[:max_preds], y_test[:max_preds]):
  pred = np.asarray(model(np.expand_dims(img, axis=0)), dtype=np.int32)
  # pred = np.float32(model(np.expand_dims(img, axis=0)))
  current_frames = []
  for i in range(pred.shape[1]):
    f = img[i, :, :, :]
    f[:, ::dy, :] = grid_color
    f[::dx, :, :] = grid_color 
    current_frames.append(np.int32(f))
  p = pred[0, -1, :, :, :]
  p[:, ::dy, :] = grid_color
  p[::dx, :, :] = grid_color 
  current_frames.append(p)
  t = truth[-1, :, :, :]
  t[:, ::dy, :] = grid_color
  t[::dx, :, :] = grid_color 
  current_frames.append(np.int32(t))
  all.append(current_frames)

In [ ]:
def img_similarity(x, y):
  d = np.absolute(np.subtract(x, y))
  return 100 - np.count_nonzero(d) * 100 / d.size

In [ ]:
img_similarity(all[0][5], all[0][8])

27.52083333333333

In [ ]:
import logging
logging.getLogger('matplotlib').setLevel(logging.ERROR)

In [ ]:
no_frame = len(all[0])
frame_w = all[0][0].shape[1]
frame_h = all[0][0].shape[0]
px = 1/plt.rcParams['figure.dpi'] 

fig, axes = plt.subplots(len(all), no_frame, figsize=(no_frame * frame_w * 2 * px, len(all) * frame_h * 2 * px))
for i in range(len(all)):
  for j in range(len(axes[i])):
    axes[i][j].imshow(np.int32(all[i][j]))
    if j < no_frame - 2:
      axes[i][j].set_title(f"Frame {j + 1}")
    elif j == no_frame - 2:
      axes[i][j].set_title(f"Prediction")
    else:
      axes[i][j].set_title(f"Truth")
